# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.


### Part II - VacationPy

Now let's use your skills in working with weather data to plan future vacations. Use Jupyter gmaps and the Google Places API for this part of the assignment.

Note: if you having trouble displaying the maps try running jupyter nbextension enable --py gmaps in your environment and retry.

Create a heat map that displays the humidity for every city from the part I of the homework.

Narrow down the DataFrame to find your ideal weather condition. For example:

A max temperature lower than 80 degrees but higher than 70.

Wind speed less than 10 mph.

Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [1]:
# !pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv('cities_df.csv')
weather_data.head()

,city,lat,lng,temp,wind speed,humidity,cloudiness,description,country
0,Jamestown,-33.21,138.60,7.69,2.69,90,0,clear sky,AU
1,Tabou,4.42,-7.36,28.19,3.97,76,29,light rain,CI
2,Saskylakh,71.97,114.09,-19.68,3.25,97,0,clear sky,RU
3,Geraldton,49.72,-86.95,2.00,1.50,100,75,broken clouds,CA
4,Chuy,-33.69,-53.46,21.99,2.73,76,100,overcast clouds,UY


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into locations
locations = weather_data[["lat", "lng"]].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=weather_data['humidity'],
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)


fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = weather_data[(weather_data['temp'] > 30) & (weather_data['cloudiness'] < 50)]
narrowed_city_df = narrowed_city_df.dropna(axis=0)
narrowed_city_df.head()

,city,lat,lng,temp,wind speed,humidity,cloudiness,description,country
10,Araouane,18.90,-3.53,32.71,6.25,8,0,clear sky,ML
34,Mana,13.03,5.27,34.06,6.07,37,20,few clouds,NG
41,Inirida,3.87,-67.92,31.94,0.89,59,19,light rain,CO
53,Tete,-16.16,33.59,43.00,5.70,13,0,clear sky,MZ
80,Kidal,18.44,1.41,37.07,1.82,10,0,clear sky,ML


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# geocoordinates
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [7]:
lat_list = []
lng_list = []
name_list = []
city_list = []
ctry_list = []

for index, row in narrowed_city_df.iterrows():
    target_coordinates = str(row['lat']) + ',' + str(row['lng'])
    
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    results = response.json()['results']
    if len(results) > 0:
        results = results[0]
        lat_list.append(results['geometry']['location']['lat'])
        lng_list.append(results['geometry']['location']['lng'])
        name_list.append(results['name'])
        city_list.append(row['city'])
        ctry_list.append(row['country'])

In [8]:
hotel_df = pd.DataFrame({
    'Hotel Name': name_list,
    'City': city_list,    
    'Country': ctry_list,
    'Lat': lat_list,
    'Lng': lng_list})

hotel_df.head()

,Hotel Name,City,Country,Lat,Lng
0,Araouane,Araouane,ML,18.904265,-3.527516
1,Shukura Coral Hotel,Mana,NG,13.038448,5.235142
2,Hotel Toninas,Inirida,CO,3.865948,-67.928594
3,Baobab Hotel Tete,Tete,MZ,-16.130210,33.618221
4,Kidal,Kidal,ML,18.452071,1.409653


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations,
                                  info_box_content=hotel_info,
                                  display_info_box=[True]*len(hotel_info))


fig.add_layer(marker_layer)


fig

# Display Map

Figure(layout=FigureLayout(height='420px'))